### CSAW output to bed
* convert CSAW output to bed file(chromosome, start, end)
* split to UP or DOWN
* with pvalue or other params

In [1]:
import os, glob
import pandas as pd

os.chdir('/Users/jplab/Desktop/DAILY_CODE_DATA/2022-4/data/4-13_ATAC_shnc_csaw/')

In [2]:
csaw_out_files = glob.glob('/Users/jplab/Desktop/DAILY_CODE_DATA/2022-4/data/4-13_ATAC_shnc_csaw/*.tsv')

## bed format descript by homer
> BED files should have at minimum 6 columns (separated by TABs, additional columns will be ignored)
* Column1: chromosome
* Column2: starting position
* Column3: ending position
* Column4: Unique Peak ID
* Column5: not used
* Column6: Strand (+/- or 0/1, where 0="+", 1="-")
In theory, HOMER will accept BED files with only 4 columns (+/- in the 4th column), and files without unique IDs, but this is NOT recommended.  For one, if you don't have unique IDs for your regions, it's hard to go back and figure out which region contains which peak.

## so I will add logcpm at col5 and up_down pvalue fdr fc at col6,7,8,9

In [8]:
for csaw_out in csaw_out_files:
    file_name = os.path.basename(csaw_out)
    file_name = file_name.replace('.tsv', '')

    # read to dataframe
    df_csaw_out = pd.read_csv(csaw_out, sep='\t',header=0)
    # filter pvalue
    df_csaw_p005 = df_csaw_out[df_csaw_out['PValue'] <= 0.05]
    # then up and down
    df_csaw_up = df_csaw_p005[df_csaw_p005['direction'] == 'up']
    df_csaw_down = df_csaw_p005[df_csaw_p005['direction'] == 'down']
    df_csaw_up.name = file_name + '_up'
    df_csaw_down.name = file_name + '_down'
    df_csaw_p005.name = file_name + '_all'
    df_csaw_out.name = file_name + '_all_noPsig'
    # put into list
    up_down_list = [df_csaw_up, df_csaw_down,df_csaw_p005,df_csaw_out]

    # save to bed file
    for df in up_down_list:
        save_as = '.p005.bed'
        if df.name.endswith('_up'):
            save_as = df.name + save_as
        if df.name.endswith('_down'):
            save_as = df.name + save_as
        if df.name.endswith('_all'):
            save_as = df.name + save_as
        if df.name.endswith('_all_noPsig'):
            save_as = df.name + '.bed'     
        # print(save_as)

        # write bed file as bed format above, at least 4 columns for homer motif analysis    
        with open(save_as, 'w') as f:
            for index, row in df.iterrows():
                f.write(
                        # chr
                        row['seqnames'] + '\t' \
                        # start
                        + str(row['start']) + '\t' \
                        # end
                        + str(row['end']) + '\t' \
                        # uniq_peak_id
                        + '{0}_{1}_{2}_{3}_{4}'.format(file_name, row['seqnames'], str(row['start']), str(row['end']), row['direction']) + '\t' \
                        # score: logcpm
                        + str(row['logCPM']) +  '\t' \
                        # strand
                        + row['strand'] + '\t' \
                        # up_down pvalue fdr fc
                        + '{0}_p{1}_q{2}_logFC{3}'.format(row['direction'], str(row['PValue']),str(row['FDR']), str(row['logFC'])) + '\n'
                    )